In [1]:
# pvcr_methods.py
# Calculates PVCR for each wafer + temperature using multiple methods.
# Units: side in µm, area in µm^2, currents in A.

import numpy as np

In [2]:


# =========================
# 1) DATA
# =========================

# Peak currents (A): side length (µm) -> I_peak (A)
data_peak = {
    "RT": {  # ~300 K
        "W20": {40: 0.020, 50: 0.033, 60: 0.043},
        "W21": {20: 0.00139, 35: 0.0043, 40: 0.0055, 50: 0.0085, 60: 0.011},
        "W22": {20: 0.00094, 30: 0.0024, 35: 0.0030, 40: 0.0040, 50: 0.0062, 60: 0.0090},
        "W65": {30: 0.017, 40: 0.023, 50: 0.035, 60: 0.046},
        "W67": {20: 0.00010, 30: 0.00028, 40: 0.00050, 50: 0.00080, 60: 0.00110},
    },
    "77K": {
        "W20": {40: 0.023, 50: 0.042, 60: 0.050},
        "W21": {35: 0.0076, 40: 0.0092, 60: 0.0190},
        "W22": {30: 0.0037, 35: 0.0041, 40: 0.0080, 50: 0.0090, 60: 0.0130},
        "W65": {30: 0.023, 40: 0.055, 50: 0.050, 60: 0.060},
        "W67": {20: 0.00017, 40: 0.00086, 50: 0.00130, 60: 0.00200},
    },
    "4.2K": {
        "W20": {},  # N/A
        "W21": {30: 0.0140},
        "W22": {30: 0.0076, 40: 0.0060, 50: 0.0082},
        "W65": {35: 0.0310},
        "W67": {20: 0.00023, 30: 0.00069, 40: 0.00120, 50: 0.00180, 60: 0.00260},
    },
}

# Valley currents (A): side length (µm) -> I_valley (A)
data_valley = {
    "RT": {
        "W20": {40: 0.010, 50: 0.017, 60: 0.034},
        "W21": {20: 0.00078, 35: 0.0025, 40: 0.0032, 50: 0.0051, 60: 0.0074},
        "W22": {20: 0.00044, 30: 0.0013, 35: 0.0014, 40: 0.0019, 50: 0.0028, 60: 0.0042},
        "W65": {30: 0.0045, 40: 0.0084, 50: 0.0133, 60: 0.0195},
        "W67": {20: 0.000078, 30: 0.00042, 40: 0.00037, 50: 0.00058, 60: 0.00083},
    },
    "77K": {
        "W20": {40: 0.0040, 50: 0.0072, 60: 0.011},
        "W21": {35: 0.00086, 40: 0.0011, 60: 0.0025},
        "W22": {30: 0.00036, 35: 0.00033, 40: 0.00041, 50: 0.00067, 60: 0.00084},
        "W65": {30: 0.0018, 40: 0.0047, 50: 0.0043, 60: 0.0062},
        "W67": {20: 0.000021, 40: 0.000093, 50: 0.00014, 60: 0.00021},
    },
    "4.2K": {
        "W20": {},  # None / N/A
        "W21": {30: 0.0010},
        "W22": {30: 0.00033, 40: 0.00027, 50: 0.00046},
        "W65": {35: 0.0017},
        "W67": {20: 0.000016, 30: 0.000045, 40: 0.000056, 50: 0.000089, 60: 0.00013},
    },
}

# =========================
# 2) FIT / SUMMARY HELPERS
# =========================

def to_arrays(side_to_I):
    """Return (A, I) sorted by side (A in µm²)."""
    if not side_to_I:
        return np.array([]), np.array([])
    sides = np.array(sorted(side_to_I.keys()), dtype=float)
    I = np.array([side_to_I[s] for s in sides], dtype=float)
    A = sides**2
    return A, I

def fit_origin(A, I):
    """Fit I = J*A (through origin). Returns J."""
    if A.size < 2:
        return np.nan
    return (A @ I) / (A @ A)

def fit_ols(A, I):
    """OLS fit I = m*A + c. Returns (m, c)."""
    if A.size < 2:
        return np.nan, np.nan
    m, c = np.polyfit(A, I, 1)
    return float(m), float(c)

def mean_J(A, I):
    """Mean of pointwise J_i = I_i/A_i."""
    if A.size == 0:
        return np.nan
    return float(np.mean(I / A))

def pvcr_per_device(peak_map, valley_map):
    """
    PVCR_i = Ipeak/Ivalley computed only where both have the same side.
    Returns (PVCR_mean, PVCR_std, n_matches, PVCR_list, sides_list)
    """
    if (not peak_map) or (not valley_map):
        return np.nan, np.nan, 0, [], []

    common_sides = sorted(set(peak_map.keys()) & set(valley_map.keys()))
    if len(common_sides) == 0:
        return np.nan, np.nan, 0, [], []

    pvcrs = []
    for s in common_sides:
        Iv = valley_map[s]
        Ip = peak_map[s]
        if Iv == 0:
            continue
        pvcrs.append(Ip / Iv)

    pvcrs = np.array(pvcrs, dtype=float)
    if pvcrs.size == 0:
        return np.nan, np.nan, 0, [], common_sides

    mean_ = float(np.mean(pvcrs))
    std_ = float(np.std(pvcrs, ddof=1)) if pvcrs.size >= 2 else 0.0
    return mean_, std_, int(pvcrs.size), pvcrs.tolist(), common_sides

# =========================
# 3) PVCR METHODS
# =========================
# Method A: PVCR from origin-fit current densities: PVCR = Jp0 / Jv0
# Method B: PVCR from OLS slopes: PVCR = mp / mv
# Method C: PVCR from mean pointwise densities: PVCR = mean(Ip/A) / mean(Iv/A)
# Method D: PVCR per-device then average: PVCR = mean( Ip/Iv ) across matched areas

def pvcr_all_methods(temp, chip):
    peak_map = data_peak.get(temp, {}).get(chip, {})
    valley_map = data_valley.get(temp, {}).get(chip, {})

    Ap, Ip = to_arrays(peak_map)
    Av, Iv = to_arrays(valley_map)

    # densities / slopes
    Jp0 = fit_origin(Ap, Ip)
    Jv0 = fit_origin(Av, Iv)
    mp, cp = fit_ols(Ap, Ip)
    mv, cv = fit_ols(Av, Iv)
    Jp_mean = mean_J(Ap, Ip)
    Jv_mean = mean_J(Av, Iv)

    # PVCR from each
    pvcr_origin = (Jp0 / Jv0) if np.isfinite(Jp0) and np.isfinite(Jv0) and Jv0 != 0 else np.nan
    pvcr_ols = (mp / mv) if np.isfinite(mp) and np.isfinite(mv) and mv != 0 else np.nan
    pvcr_meanJ = (Jp_mean / Jv_mean) if np.isfinite(Jp_mean) and np.isfinite(Jv_mean) and Jv_mean != 0 else np.nan

    pvcr_dev_mean, pvcr_dev_std, nmatch, pvcr_list, common_sides = pvcr_per_device(peak_map, valley_map)

    return {
        "temp": temp,
        "chip": chip,
        "n_peak": int(len(peak_map)),
        "n_valley": int(len(valley_map)),
        "PVCR_originJ": pvcr_origin,
        "PVCR_olsSlope": pvcr_ols,
        "PVCR_meanJ": pvcr_meanJ,
        "PVCR_dev_mean": pvcr_dev_mean,
        "PVCR_dev_std": pvcr_dev_std,
        "PVCR_dev_n": nmatch,
        "common_sides": common_sides,
        # extra diagnostics (optional)
        "Jp0": Jp0, "Jv0": Jv0,
        "mp": mp, "cp": cp,
        "mv": mv, "cv": cv,
        "Jp_mean": Jp_mean, "Jv_mean": Jv_mean,
    }

# =========================
# 4) RUN + PRINT SUMMARY
# =========================

temps = ["RT", "77K", "4.2K"]
chips = ["W20", "W21", "W22", "W65", "W67"]

print("\n=== PVCR by multiple methods ===")
for temp in temps:
    print(f"\n--- {temp} ---")
    for chip in chips:
        out = pvcr_all_methods(temp, chip)

        # Only print if we have at least one peak+valley point
        if out["n_peak"] == 0 or out["n_valley"] == 0:
            continue

        print(
            f"{chip}: "
            f"PVCR(origin J)={out['PVCR_originJ']:.3g} | "
            f"PVCR(OLS slope)={out['PVCR_olsSlope']:.3g} | "
            f"PVCR(mean I/A)={out['PVCR_meanJ']:.3g} | "
            f"PVCR(mean Ip/Iv over matched areas)={out['PVCR_dev_mean']:.3g}"
            + (f" ± {out['PVCR_dev_std']:.2g} (n={out['PVCR_dev_n']})" if out["PVCR_dev_n"] else "")
        )

# If you want to inspect one case in detail:
# temp_sel, chip_sel = "RT", "W67"
# print(pvcr_all_methods(temp_sel, chip_sel))


=== PVCR by multiple methods ===

--- RT ---
W20: PVCR(origin J)=1.49 | PVCR(OLS slope)=0.941 | PVCR(mean I/A)=1.67 | PVCR(mean Ip/Iv over matched areas)=1.74 ± 0.41 (n=3)
W21: PVCR(origin J)=1.58 | PVCR(OLS slope)=1.46 | PVCR(mean I/A)=1.67 | PVCR(mean Ip/Iv over matched areas)=1.67 ± 0.11 (n=5)
W22: PVCR(origin J)=2.14 | PVCR(OLS slope)=2.21 | PVCR(mean I/A)=2.09 | PVCR(mean Ip/Iv over matched areas)=2.1 ± 0.13 (n=6)
W65: PVCR(origin J)=2.52 | PVCR(OLS slope)=1.99 | PVCR(mean I/A)=2.86 | PVCR(mean Ip/Iv over matched areas)=2.88 ± 0.62 (n=4)
W67: PVCR(origin J)=1.3 | PVCR(OLS slope)=1.53 | PVCR(mean I/A)=1.11 | PVCR(mean Ip/Iv over matched areas)=1.2 ± 0.3 (n=5)

--- 77K ---
W20: PVCR(origin J)=5.03 | PVCR(OLS slope)=3.79 | PVCR(mean I/A)=5.34 | PVCR(mean Ip/Iv over matched areas)=5.38 ± 0.72 (n=3)
W21: PVCR(origin J)=7.82 | PVCR(OLS slope)=6.97 | PVCR(mean I/A)=8.27 | PVCR(mean Ip/Iv over matched areas)=8.27 ± 0.62 (n=3)
W22: PVCR(origin J)=14.9 | PVCR(OLS slope)=16.9 | PVCR(mean I/

In [3]:
#choose PVCR matched and PVCR origin J

In [4]:
import numpy as np

def origin_slope_and_se(A, I):
    """
    Fit I = J*A through origin.
    Returns J_hat, SE(J_hat), residuals.
    """
    A = np.asarray(A, float)
    I = np.asarray(I, float)
    n = A.size
    if n < 2:
        return np.nan, np.nan, np.array([])

    J = (A @ I) / (A @ A)
    resid = I - J*A

    # dof = n - 1 (one parameter fitted)
    dof = n - 1
    s2 = np.sum(resid**2) / dof if dof > 0 else np.nan
    se = np.sqrt(s2 / np.sum(A**2)) if np.isfinite(s2) else np.nan
    return J, se, resid

def bootstrap_origin_slope(A, I, nboot=5000, seed=0):
    """
    Bootstrap uncertainty for origin slope.
    Returns (J_hat, J_std, (lo, hi) 95% CI).
    """
    rng = np.random.default_rng(seed)
    A = np.asarray(A, float)
    I = np.asarray(I, float)
    n = A.size
    if n < 2:
        return np.nan, np.nan, (np.nan, np.nan)

    Jhat = (A @ I) / (A @ A)
    Js = np.empty(nboot, float)
    idx = np.arange(n)
    for b in range(nboot):
        samp = rng.choice(idx, size=n, replace=True)
        Ab = A[samp]
        Ib = I[samp]
        Js[b] = (Ab @ Ib) / (Ab @ Ab)

    Jstd = float(np.std(Js, ddof=1))
    lo, hi = np.percentile(Js, [2.5, 97.5])
    return float(Jhat), Jstd, (float(lo), float(hi))

def pvcr_from_J_with_error(Jp, seJp, Jv, seJv):
    """
    PVCR = Jp/Jv with approximate propagated uncertainty (independent errors).
    Returns PVCR, SE(PVCR).
    """
    if not np.isfinite(Jp) or not np.isfinite(Jv) or Jv == 0:
        return np.nan, np.nan
    pvcr = Jp / Jv
    if (not np.isfinite(seJp)) or (not np.isfinite(seJv)) or Jp == 0:
        return pvcr, np.nan
    # error propagation for ratio:
    # (σ_pvcr / pvcr)^2 ≈ (σ_Jp/Jp)^2 + (σ_Jv/Jv)^2
    se = abs(pvcr) * np.sqrt((seJp/Jp)**2 + (seJv/Jv)**2)
    return pvcr, se

# Example usage for one chip/temp (edit these):
TEMP = "RT"
CHIP = "W67"

# Pull peak + valley arrays
def to_A_I(d):
    if not d:
        return np.array([]), np.array([])
    sides = np.array(sorted(d.keys()), float)
    A = sides**2
    I = np.array([d[s] for s in sides], float)
    return A, I

Ap, Ip = to_A_I(data_peak[TEMP][CHIP])
Av, Iv = to_A_I(data_valley[TEMP][CHIP])

Jp, seJp, _ = origin_slope_and_se(Ap, Ip)
Jv, seJv, _ = origin_slope_and_se(Av, Iv)
pvcr, se_pvcr = pvcr_from_J_with_error(Jp, seJp, Jv, seJv)

print(f"{CHIP} @ {TEMP}")
print(f"  J_peak(origin)  = {Jp:.3e} ± {seJp:.2e}  A/µm² (1σ)")
print(f"  J_valley(origin)= {Jv:.3e} ± {seJv:.2e}  A/µm² (1σ)")
print(f"  PVCR_J = {pvcr:.3g} ± {se_pvcr:.2g} (approx 1σ)")

# Optional: bootstrap CI for J_peak and J_valley
Jp_hat, Jp_std, (Jp_lo, Jp_hi) = bootstrap_origin_slope(Ap, Ip, nboot=5000, seed=1)
Jv_hat, Jv_std, (Jv_lo, Jv_hi) = bootstrap_origin_slope(Av, Iv, nboot=5000, seed=2)
print(f"  Bootstrap J_peak:  std={Jp_std:.2e}, 95% CI=({Jp_lo:.3e}, {Jp_hi:.3e})")
print(f"  Bootstrap J_valley: std={Jv_std:.2e}, 95% CI=({Jv_lo:.3e}, {Jv_hi:.3e})")

W67 @ RT
  J_peak(origin)  = 3.101e-07 ± 4.03e-09  A/µm² (1σ)
  J_valley(origin)= 2.392e-07 ± 2.19e-08  A/µm² (1σ)
  PVCR_J = 1.3 ± 0.12 (approx 1σ)
  Bootstrap J_peak:  std=4.42e-09, 95% CI=(3.045e-07, 3.186e-07)
  Bootstrap J_valley: std=2.48e-08, 95% CI=(2.299e-07, 3.134e-07)


In [5]:
# =========================
# DROP-IN ADDITION (put straight under your existing bootstrap J_peak/J_valley prints)
# Uses the bootstrap std + CI you already computed (Jp_hat, Jp_std, Jp_lo/hi, etc.)
# to estimate PVCR_J uncertainty.
# =========================

# Point estimate from the bootstrap point estimates (should match your pvcr_hat closely)
pvcr_boot_hat = Jp_hat / Jv_hat

# 1σ uncertainty via standard error propagation (treating Jp and Jv as independent)
pvcr_boot_std = abs(pvcr_boot_hat) * np.sqrt((Jp_std / Jp_hat)**2 + (Jv_std / Jv_hat)**2)

# 95% CI via combining bootstrap percentile bounds (conservative, not exact but OK)
pvcr_lo = Jp_lo / Jv_hi   # smallest ratio
pvcr_hi = Jp_hi / Jv_lo   # largest ratio

print(f"  Bootstrap PVCR_J: {pvcr_boot_hat:.3g} ± {pvcr_boot_std:.2g} (1σ), 95% CI≈({pvcr_lo:.3g}, {pvcr_hi:.3g})")

  Bootstrap PVCR_J: 1.3 ± 0.14 (1σ), 95% CI≈(0.972, 1.39)


In [6]:
# =========================
# DROP-IN CELL: PVCR (origin J) + uncertainty, for ALL wafers & temperatures
# - Computes J_peak and J_valley from through-origin fit I = J*A
# - Computes SE(J) from residual scatter (1σ)
# - Computes PVCR_J = J_peak / J_valley
# - Computes propagated 1σ uncertainty on PVCR_J from SE(J_peak), SE(J_valley)
# - Optional: bootstrap PVCR_J std + 95% CI (set DO_BOOTSTRAP=True)
#
# Assumes you already have: data_peak, data_valley
# Data format: data_peak[temp][chip] = {side_um: I_peak_A, ...}
# =========================

import numpy as np

DO_BOOTSTRAP = True
NBOOT = 5000
BOOT_SEED = 123

def to_A_I(side_to_I):
    if not side_to_I:
        return np.array([]), np.array([])
    sides = np.array(sorted(side_to_I.keys()), float)
    A = sides**2  # µm^2
    I = np.array([side_to_I[s] for s in sides], float)
    return A, I

def origin_J_and_se(A, I):
    """
    Fit I = J*A through origin.
    Returns (J_hat, SE(J_hat)).
    SE uses dof=n-1 and assumes roughly constant scatter in I.
    """
    A = np.asarray(A, float)
    I = np.asarray(I, float)
    n = A.size
    if n < 2:
        return np.nan, np.nan

    J = (A @ I) / (A @ A)
    resid = I - J*A

    dof = n - 1
    s2 = np.sum(resid**2) / dof if dof > 0 else np.nan
    se = np.sqrt(s2 / np.sum(A**2)) if np.isfinite(s2) and np.sum(A**2) > 0 else np.nan
    return float(J), float(se)

def pvcr_and_se(Jp, seJp, Jv, seJv):
    """
    PVCR = Jp/Jv with approximate propagated 1σ uncertainty.
    """
    if not np.isfinite(Jp) or not np.isfinite(Jv) or Jv == 0:
        return np.nan, np.nan
    pvcr = Jp / Jv
    if (not np.isfinite(seJp)) or (not np.isfinite(seJv)) or Jp == 0 or Jv == 0:
        return float(pvcr), np.nan
    se = abs(pvcr) * np.sqrt((seJp/Jp)**2 + (seJv/Jv)**2)
    return float(pvcr), float(se)

def bootstrap_pvcr_origin(Ap, Ip, Av, Iv, nboot=5000, seed=0):
    """
    Bootstrap PVCR_J where PVCR_J = (Jp0/Jv0), each from origin-fit.
    Resamples peak points and valley points independently (with replacement).
    Returns (pvcr_std, pvcr_lo95, pvcr_hi95).
    """
    rng = np.random.default_rng(seed)

    Ap = np.asarray(Ap, float); Ip = np.asarray(Ip, float)
    Av = np.asarray(Av, float); Iv = np.asarray(Iv, float)

    npk, nvl = Ap.size, Av.size
    if npk < 2 or nvl < 2:
        return np.nan, np.nan, np.nan

    idxp = np.arange(npk)
    idxv = np.arange(nvl)

    pvcrs = np.empty(nboot, float)
    for b in range(nboot):
        sp = rng.choice(idxp, size=npk, replace=True)
        sv = rng.choice(idxv, size=nvl, replace=True)

        Abp, Ibp = Ap[sp], Ip[sp]
        Abv, Ibv = Av[sv], Iv[sv]

        Jp = (Abp @ Ibp) / (Abp @ Abp)
        Jv = (Abv @ Ibv) / (Abv @ Abv)

        pvcrs[b] = Jp / Jv if Jv != 0 else np.nan

    pvcrs = pvcrs[np.isfinite(pvcrs)]
    if pvcrs.size < 2:
        return np.nan, np.nan, np.nan

    pvcr_std = float(np.std(pvcrs, ddof=1))
    lo, hi = np.percentile(pvcrs, [2.5, 97.5])
    return pvcr_std, float(lo), float(hi)

# Build list of all temp/chip combos present in either dataset
temps = sorted(set(data_peak.keys()) | set(data_valley.keys()))
chips = sorted(set(
    [c for t in data_peak for c in data_peak[t].keys()] +
    [c for t in data_valley for c in data_valley[t].keys()]
))

print("\n=== PVCR from origin-fit J, with uncertainties ===")
print("(J in A/µm²; PVCR dimensionless)\n")

for temp in temps:
    print(f"--- {temp} ---")
    for chip in chips:
        peak_map = data_peak.get(temp, {}).get(chip, {})
        valley_map = data_valley.get(temp, {}).get(chip, {})

        Ap, Ip = to_A_I(peak_map)
        Av, Iv = to_A_I(valley_map)

        # Need at least 2 points in EACH to fit J with SE
        if Ap.size < 2 or Av.size < 2:
            # still allow a single-point PVCR estimate if you want:
            # (comment this block out if you prefer to skip)
            if Ap.size >= 1 and Av.size >= 1:
                # pick matched side if possible; otherwise smallest available in each
                # (this is just a fallback; fits are preferred)
                Jp_single = float(Ip[0] / Ap[0])
                Jv_single = float(Iv[0] / Av[0])
                pvcr_single = Jp_single / Jv_single if Jv_single != 0 else np.nan
                print(f"{chip}: insufficient points for fit (n_peak={Ap.size}, n_valley={Av.size}) | PVCR≈{pvcr_single:.3g} (single-point)")
            continue

        Jp, seJp = origin_J_and_se(Ap, Ip)
        Jv, seJv = origin_J_and_se(Av, Iv)
        pvcr, se_pvcr = pvcr_and_se(Jp, seJp, Jv, seJv)

        if DO_BOOTSTRAP:
            pvcr_boot_std, pvcr_lo, pvcr_hi = bootstrap_pvcr_origin(
                Ap, Ip, Av, Iv, nboot=NBOOT, seed=(BOOT_SEED + hash((temp, chip)) % 10_000)
            )
            print(
                f"{chip}: PVCR_J={pvcr:.3g} ± {se_pvcr:.2g} (prop 1σ) | "
                f"bootstrap: ±{pvcr_boot_std:.2g} (1σ), 95% CI=({pvcr_lo:.3g}, {pvcr_hi:.3g})"
            )
        else:
            print(f"{chip}: PVCR_J={pvcr:.3g} ± {se_pvcr:.2g} (prop 1σ)")
    print()


=== PVCR from origin-fit J, with uncertainties ===
(J in A/µm²; PVCR dimensionless)

--- 4.2K ---
W21: insufficient points for fit (n_peak=1, n_valley=1) | PVCR≈14 (single-point)
W22: PVCR_J=19.7 ± 6.3 (prop 1σ) | bootstrap: ±5.9 (1σ), 95% CI=(11.8, 43.1)
W65: insufficient points for fit (n_peak=1, n_valley=1) | PVCR≈18.2 (single-point)
W67: PVCR_J=19.9 ± 0.77 (prop 1σ) | bootstrap: ±0.76 (1σ), 95% CI=(17.7, 20.7)

--- 77K ---
W20: PVCR_J=5.03 ± 0.38 (prop 1σ) | bootstrap: ±0.38 (1σ), 95% CI=(4.61, 6.05)
W21: PVCR_J=7.82 ± 0.3 (prop 1σ) | bootstrap: ±0.37 (1σ), 95% CI=(7.6, 8.92)
W22: PVCR_J=14.9 ± 1.3 (prop 1σ) | bootstrap: ±1.3 (1σ), 95% CI=(12.5, 17.6)
W65: PVCR_J=10.7 ± 2.1 (prop 1σ) | bootstrap: ±2.3 (1σ), 95% CI=(7.29, 17.4)
W67: PVCR_J=9.42 ± 0.22 (prop 1σ) | bootstrap: ±0.26 (1σ), 95% CI=(8.93, 9.82)

--- RT ---
W20: PVCR_J=1.49 ± 0.18 (prop 1σ) | bootstrap: ±0.22 (1σ), 95% CI=(1.31, 2)
W21: PVCR_J=1.58 ± 0.046 (prop 1σ) | bootstrap: ±0.063 (1σ), 95% CI=(1.51, 1.71)
W22: PVCR_